In [1]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt


In [2]:
import os
cascPath = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_default.xml"
cascPath2 = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_mcs_mouth.xml"

In [3]:
face_cascade = cv2.CascadeClassifier(cascPath)
#eye_cascade = cv2.CascadeClassifier('data\\xml\\haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cascPath2)
#upper_body = cv2.CascadeClassifier('data\\xml\\haarcascade_upperbody.xml')


In [4]:
bw_threshold = 80

In [5]:
#for message to be displayed to user

font = cv2.FONT_HERSHEY_SIMPLEX
#what is org for?
org = (30, 30)
weared_mask_font_color = (255, 255, 255)
not_weared_mask_font_color = (0, 0, 255)
thickness = 2
font_scale = 1
weared_mask = "Thank You for wearing MASK"
not_weared_mask = "Please wear MASK to defeat Corona"

In [6]:
# 0 is for camera preference of laptop webcam, can add external vid source here
cap = cv2.VideoCapture('mask.mp4')

In [7]:
while 1:
    # Get individual frame
    ret, img = cap.read()
    img = cv2.flip(img,1)
    

    # Convert Image into gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

    # Convert image in black and white
    (thresh, black_and_white) = cv2.threshold(gray, bw_threshold, 255, cv2.THRESH_BINARY)
    #cv2.imshow('black_and_white', black_and_white)

    # detect face
    #faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    faces = face_cascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)    


    # Face prediction for black and white
    faces_bw = face_cascade.detectMultiScale(black_and_white, 1.1, 4)

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
if(len(faces) == 0 and len(faces_bw) == 0):
    cv2.putText(img, "No face found...", org, font, font_scale, weared_mask_font_color, thickness, cv2.LINE_AA)
        
elif(len(faces) == 0 and len(faces_bw) == 1):
    # It has been observed that for white mask covering mouth, with gray image face prediction is not happening
    cv2.putText(img, weared_mask, org, font, font_scale, weared_mask_font_color, thickness, cv2.LINE_AA)
else:
    # Draw rectangle on face
    for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), 2)
    roi_gray = gray[y:y + h, x:x + w]
    roi_color = img[y:y + h, x:x + w]


In [ ]:
# Detect lips counters
mouth_rects = mouth_cascade.detectMultiScale(gray, 1.5, 5)


In [ ]:
# Face detected but Lips not detected which means person is wearing mask
if(len(mouth_rects) == 0):
    cv2.putText(img, weared_mask, org, font, font_scale, weared_mask_font_color, thickness, cv2.LINE_AA)
else:
    for (mx, my, mw, mh) in mouth_rects:

        if(y < my < y + h):
            # Face and Lips are detected but lips coordinates are within face cordinates which `means lips prediction is true and
            # person is not waring mask
            cv2.putText(img, not_weared_mask, org, font, font_scale, not_weared_mask_font_color, thickness, cv2.LINE_AA)

            #cv2.rectangle(img, (mx, my), (mx + mh, my + mw), (0, 0, 255), 3)
            break

In [ ]:
# Show frame with results
cv2.imshow('Mask Detection', img)
k = cv2.waitKey(30) & 0xff
if k == 27:
    break

In [ ]:
# Release video
cap.release()
cv2.destroyAllWindows()